# Neurorobotics with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [ ]:
import time
import numpy as np
import csv
import random

import subprocess
import re

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

In [ ]:
def isFinished(processes):
    
    for process in processes:
        if(process.poll() == None):
            return False
    
    return True

In [ ]:
output_min = []
error_min = 200
number_params = 4
number_generations = 20

# hyperparameters
npop = 50 # population size
sigma = 0.1 # noise standard deviation
alpha = 0.01 # learning rate

# our initial guess is random
w = np.random.uniform(low=-4, high=-2, size=(1,number_params))

print('w_start: %s' % (str(w)))

print("Starting")

# start the optimization
for generation in range(number_generations):
    
    # Start time measurement
    start = time.time()
    
    weights_list = []
    processes = []
    
    # Initialize memory for a population of w's, and their rewards
    N = np.random.randn(npop, number_params) # samples from a normal distribution N(0,1)
    R = np.zeros(npop)
    
    # Execute all subprocesses and save outputs into files
    for i in range(npop):
        w_try = w + sigma*N[i] # jitter w using gaussian of sigma 0.1
        weights = []
        for j in range(number_params):
            weights.append( str(10**w_try[0,j]) ) 

        # Spawn the subprocesses and write outputs into files
        filename = "temp/log" + str(i) + ".txt"
        with open(filename, "w") as file:
            command = ['python', 'OptimizationJob_SmallBrain.py']
            command.extend(weights)
            processes.append( subprocess.Popen(command, stdout=file) ) 
            
        # Append current weights to weights list
        weights_list.append(weights)
            
    # Wait until all subprocesses are finished
    while isFinished(processes) == False:
        time.sleep(0.5)
    
    # Read outputs
    for i in range(npop):

        with open("temp/log" + str(i) + ".txt") as file:
            filetext = file.read()

            # Find fitness value through regex
            matches = re.findall("(.*[^=])=(.*)", filetext)
            error = float(matches[0][1])
            R[j] = error
            
            #print([weights_list[i], error])
            
            # A new minimum is found
            if error < error_min:
                error_min = error
                print(weights_list[i])
                print(error)

                
    # standardize the rewards to have a gaussian distribution
    A = (R - np.mean(R)) / np.std(R)

    # perform the parameter update. The matrix multiply below
    # is just an efficient way to sum up all the rows of the noise matrix N,
    # where each row N[j] is weighted by A[j]
    w = w + alpha/(npop*sigma) * np.dot(N.T, A)
                
    # Time measurement
    end = time.time()
    
    # Ausgabe
    print('Time taken in seconds -', end - start)
    
    # print current fitness of the most likely parameter setting
    print('iter %d. w: %s' % (generation, str(w)))
    
print("Finished")

### Time measurement for small brain

| Number of subprocesses   |  Elapsed Time   |  Elapsed Time per subprocess  |
|--------------------------|-----------------|-------------------------------|
| 1                        |    39.06 s      |         39.06 s               |
| 5                        |    42.63 s      |          8.52 s               |
| 10                       |    44.72 s      |          4.47 s               |
| 20                       |    49.58 s      |          2.48 s               |
| 30                       |    66.34 s      |          2.21 s               |
| 40                       |    73.10 s      |          1.83 s               |
| 50                       |    92.97 s      |          1.86 s               |
| 80                       |   149.80 s      |          1.87 s               |
|100                       |   188.15 s      |          1.88 s               |
|200                       |   382.65 s      |          1.91 s               |
|300                       |   592.33 s      |          1.97 s               |
|500                       |   993.66 s      |          1.99 s               |
|800                       |  1599.73 s      |          1.99 s               |